### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return ((1 - np.sum((l_c / l_s) ** 2,
                            axis=1).reshape(-1, 1))
                * l_s + (1 - np.sum((r_c / r_s) ** 2,
                                    axis=1).reshape(-1, 1))
                * r_s) / (l_s + r_s)

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -(np.sum((l_c / l_s) * np.log2((l_c + 1) / l_s),
                        axis=1).reshape(-1, 1)
                 * l_s + np.sum((r_c / r_s) * np.log2((r_c + 1) / r_s),
                                axis=1).reshape(-1, 1)
                 * r_s) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return ((1 - np.max(l_c / l_s,
                            axis=1).reshape(-1, 1))
                * l_s + (1 - np.max(r_c / r_s,
                                    axis=1).reshape(-1, 1))
                * r_s) / (l_s + r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        x_sorted, y_sorted = self.__sort_samples(x, y)
        cut_size = np.int(self.min_samples_split / 2 - 1)
        if cut_size == 0:
            y_sorted_split = y_sorted
        else:
            y_sorted_split = y_sorted[cut_size:-cut_size]
        r_border_ids = np.where(y_sorted_split[:-1] !=
                                y_sorted_split[1:])[0] + (cut_size + 1)
        if len(r_border_ids) == 0:
            return np.inf, np.inf
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        one_hot_encode = np.zeros((r_border_ids.shape[0], self.num_class))
        one_hot_encode[np.arange(r_border_ids.shape[0]),
                       y_sorted[r_border_ids - 1]] = 1
        class_increments = one_hot_encode * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
            np.bincount(y_sorted[:cut_size], minlength=self.num_class)
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(y_sorted,
                                    minlength=self.num_class) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = y_sorted.shape[0] - l_sizes
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
        left_el_id = l_sizes[idx][0]
        return gs[idx][0], (x_sorted[left_el_id-1] +
                            x_sorted[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth):
        yn = np.bincount(y, minlength=self.num_class)
        pr = yn / len(y)

        if depth == self.max_depth or np.unique(y).size == 1 or \
                np.bincount(y).max() >= (y.size * self.sufficient_share) \
                or y.size <= self.min_samples_split:
            self.tree[node_id] = (self.LEAF_TYPE, yn.argmax(), pr)
            return

        f_ids = self.get_feature_ids(x.shape[1])
        thr = np.empty(x.shape[1])
        gss = np.empty(x.shape[1])
        for i in f_ids:
            gss[i], thr[i] = self.__find_threshold(x[:, i], y)
        top_f_id = gss.argmin()
        top_thr = thr[top_f_id]
        l_x, r_x, l_y, r_y = self.__div_samples(x, y, top_f_id, top_thr)

        if l_y.size == 0 or r_y.size == 0:
            self.tree[node_id] = (self.LEAF_TYPE, yn.argmax(), pr)
            return

        l_n = np.bincount(l_y, minlength=self.num_class)
        r_n = np.bincount(r_y, minlength=self.num_class)
        p = l_y.size / (l_y.size + r_y.size)
        q = r_y.size / (l_y.size + r_y.size)
        self.feature_importances_[top_f_id] = (1 - ((yn / y.size) ** 2).sum())
        -p * (1 - ((l_n / l_y.size) ** 2).sum())
        -q * (1 - ((r_n / r_y.size) ** 2).sum())

        self.tree[node_id] = (self.NON_LEAF_TYPE, top_f_id, top_thr)
        self.__fit_node(l_x, l_y, node_id * 2 + 1, depth + 1)
        self.__fit_node(r_x, r_y, node_id * 2 + 2, depth + 1)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

Wall time: 1.01 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

Wall time: 7.95 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8917748917748917

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8917748917748917

## Подготовка данных Speed Dating Data 

In [9]:
#%%pycodestyle
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')
df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
              'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin',
              'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
              'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o',
              'shar_o', 'like_o',  'prob_o', 'met_o', 'field', 'undergra',
              'imprelig', 'imprace', 'from', 'zipcode', 'career', 'sports',
              'tvsports', 'exercise', 'dining', 'museums',
              'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
              'theater', 'movies', 'concerts', 'music', 'shopping',
              'yoga', 'expnum', 'wave'], axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
pd.get_dummies(df, columns=['field_cd'], prefix='field_cd', prefix_sep='=')
pd.get_dummies(df, columns=['race'], prefix='race', prefix_sep='=')
df.loc[:, 'mn_sat'] = \
    df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = \
    df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.tuition.fillna(-999)
df.loc[:, 'income'] = \
    df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df.loc[:, 'temp_totalsum'] = \
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
               'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
           'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                     'fun1_1', 'amb1_1', 'shar1_1']].T /
                          df.loc[:, 'temp_totalsum'].T).T * 100
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                        'fun2_1', 'amb2_1', 'shar2_1']] \
                                   .sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
           'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                     'fun2_1', 'amb2_1', 'shar2_1']].T /
                          df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
            'intel{}_1'.format(i), 'fun{}_1'.format(i),
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    df = df.drop(feat, axis=1)

df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                .drop(['gender'], axis=1).dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                  .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                  .dropna()
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [11]:
%time clf.fit(X_train, y_train)

Wall time: 62.9 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [12]:
%time my_clf.fit(X_train, y_train)

Wall time: 402 ms


## Проверка качества работы на Speed Dating Data

In [13]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5547754612783081

In [14]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5570321151716501

## Задание 3

In [15]:
#

## Задание 4

In [16]:
#%%pycodestyle
pd.Series(data=my_clf.feature_importances_,
          index=df_pair.columns[1:]).sort_values(ascending=False).head(10)

samerace      0.500000
age           0.500000
amb1_1        0.497778
intel1_1      0.495868
tuition_f     0.489796
mn_sat        0.487535
intel2_1      0.486852
amb2_1        0.483741
attr3_1_f     0.483471
exphappy_f    0.480000
dtype: float64

In [17]:
#%%pycodestyle
pd.Series(data=clf.feature_importances_,
           index=df_pair.columns[1:]).sort_values(ascending=False).head(10)

int_corr      0.083479
attr1_1_f     0.033382
shar1_1_f     0.033237
sinc1_1       0.028179
fun2_1_f      0.025500
field_cd      0.025392
attr2_1       0.024915
field_cd_f    0.024035
shar2_1_f     0.022684
intel3_1_f    0.022680
dtype: float64

## Задание 5

In [18]:
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=10)

In [19]:
#%%pycodestyle
params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 10),
    'min_samples_split': range(2, 16, 5)
}

CV = GridSearchCV(estimator=rfc, param_grid=params)
CV.fit(X, y)
print(CV.best_params_)
print(CV.best_score_)

C:\Users\mserg\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'criterion': 'gini', 'max_depth': 1, 'min_samples_split': 2}
0.8364591147786947
